In [1]:

import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv(
    'https://raw.githubusercontent.com/numenta'
    '/NAB/master/data/realKnownCause/ambient'
    '_temperature_system_failure.csv')

# Exclude datetime column
data_values = data.drop('timestamp',
                        axis=1).values

# Convert data to float type
data_values = data_values.astype('float32')

# Create new dataframe with converted values
data_converted = pd.DataFrame(data_values,
                              columns=data.columns[1:])

# Add back datetime column
data_converted.insert(0, 'timestamp',
                      data['timestamp'])

In [3]:

data_converted = data_converted.dropna()

In [4]:
#Anomaly Detection using Autoencoder
# It is a type of neural network that learns to compress and then reconstruct the original data, allowing it to identify anomalies in the data.

In [5]:
# Exclude datetime column again
data_tensor = tf.convert_to_tensor(data_converted.drop(
    'timestamp', axis=1).values, dtype=tf.float32)

# Define the autoencoder model
input_dim = data_converted.shape[1] - 1
encoding_dim = 10

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='relu')(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compile and fit the model
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(data_tensor, data_tensor, epochs=50,
                batch_size=32, shuffle=True)

# Calculate the reconstruction error for each data point
reconstructions = autoencoder.predict(data_tensor)
mse = tf.reduce_mean(tf.square(data_tensor - reconstructions),
                     axis=1)
anomaly_scores = pd.Series(mse.numpy(), name='anomaly_scores')
anomaly_scores.index = data_converted.index


threshold = anomaly_scores.quantile(0.99)
anomalous = anomaly_scores > threshold
binary_labels = anomalous.astype(int)
precision, recall,\
    f1_score, _ = precision_recall_fscore_support(
        binary_labels, anomalous, average='binary')



test = data_converted['value'].values
predictions = anomaly_scores.values

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1_score)

Epoch 1/50
228/228 [==============================] - 2s 3ms/step - loss: 5093.5220
Epoch 2/50
228/228 [==============================] - 1s 6ms/step - loss: 5093.5229
Epoch 3/50
228/228 [==============================] - 2s 9ms/step - loss: 5093.5249
Epoch 4/50
228/228 [==============================] - 1s 3ms/step - loss: 5093.5239
Epoch 5/50
228/228 [==============================] - 1s 3ms/step - loss: 5093.5239
Epoch 6/50
228/228 [==============================] - 1s 3ms/step - loss: 5093.5239
Epoch 7/50
228/228 [==============================] - 1s 3ms/step - loss: 5093.5225
Epoch 8/50
228/228 [==============================] - 1s 4ms/step - loss: 5093.5249
Epoch 9/50
228/228 [==============================] - 1s 3ms/step - loss: 5093.5220
Epoch 10/50
228/228 [==============================] - 1s 4ms/step - loss: 5093.5215
Epoch 11/50
228/228 [==============================] - 1s 4ms/step - loss: 5093.5234
Epoch 12/50
228/228 [==============================] - 1s 4ms/step - loss: